### Introduction to Low-Level Image Processing

You can get a long way in image processing and computer vision with
fairly simple operations that basically just work with arithmetic.
They're not only easier to use than more complex algorithms, but 
also usually faster, easier to inspect, and more reliable than 
complex algorithms. 

It's good to understand low-level techniques if you think you might want 
to use more complex techniques for image recognition or processing, 
including machine learning methods, because complex techniques often 
don't work well without preprocessing by simpler functions. Simple 
functions can also be assembled into quite complex ones!

In those notebook, we'll primarily be using "morphological" operations 
-- operations that work with line, shape, and form. However, we'll start
with a little introduction to their extended family.

#### Footprint Operations

Many image manipulation techniques rely on looking at each pixel in the 
image, then applying some kind of mathematical function to the pixels 
that fall within some region around it. This region is called a "footprint"
(or sometimes a "kernel").  

Let's look briefly at a couple of _non_-morphological examples of this
kind of technique: the median and maximum filters. These filters do pretty
much what their names suggest: they replace each pixel with the median or 
maximum value of the surrounding pixels within a footprint.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# NOTE: I don't remember if we opened 'desktop' images before.
from PIL import Image
import scipy.ndimage as ndi
# This is a little settings file that removes all the annoying
# borders and axes and things that aren't really relevant
# when we're just using matplotlib to look at things as images.
plt.style.use('settings/simple_image.mplstyle')

In [ ]:
# load in a noisy cross shape.
noisecross = np.asarray(Image.open('images/noisecross.png'))
_ = plt.imshow(noisecross)

In [ ]:
# generate big and small footprints.
# note that what's 'big' and 'small' depends on the
# size of the image you're applying a filter to!
fbig, fsmall = 10, 3
big_foot, small_foot = np.ones((fbig, fbig)), np.ones((fsmall, fsmall))
# these are just arrays:
small_foot

In [ ]:
# The median filter has a 'softening' effect. When the footprint is small, the 
# softening is relatively mild; as it gets bigger, the softening gets more intense.
# This is a little bit like turning up the bass on a piece of music: it emphasizes
# more consistent parts of the image and suppresses more variable ones.
fig, axes = plt.subplots(1, 3, figsize=(10, 4))
axes[0].imshow(noisecross)
axes[1].imshow(ndi.median_filter(noisecross, footprint=small_foot))
axes[2].imshow(ndi.median_filter(noisecross, footprint=big_foot))
for i, title in zip((0, 1, 2), ("original", "median_small", "median_big")):
    axes[i].set_title(title)

In [ ]:
# This can be used for many practical purposes. For instance, look 
# at what a median filter can do to the scanlines in this classic Viking Orbiter image
# (a technique the ground team used to great advantage!):
import pdr
viking = pdr.read("/datascratch/viking/edr/vo_1023/f611axx/F611A13.IMG").IMAGE
fig, axes = plt.subplots(1, 2, figsize=(15, 9))
axes[0].imshow(viking)
axes[1].imshow(ndi.median_filter(viking, footprint=np.ones((4, 4))))

### Making Binary Images

Because subjective fullness of the Moon is basically about shape and line,
we'd like to be able to work directly with those aspects of images. 
Morphological operators are powerful tools for doing this. Morphological
operators are a type of footprint-based filter, that use logical operations
like "and" and "or" instead of arithmetic.

This means that they want to work on "binary" images -- black and white 
images made up of only 1s and 0s.

Most images we want to work with don't start out as binary images. The
easiest (and one of the most effective) ways to reduce images to lines
is to set a cutoff value or "threshold". We then set all values below
that threshold to black, and all values above that threshold to white. 
If it's a color image, we also want to turn it to grayscale first.
This often works something like tracing or making an outline of an image --
and these are also good first steps in other processes that want outlines,
like silkscreening.

Let's go ahead and walk through the process.

In [ ]:
# Here is a detail of part of a Tiffany lamp.
tiffany = np.asarray(Image.open("images/tiffany.png"))
plt.imshow(tiffany)

In [ ]:
# Color images are usually 3-D arrays where the third axis
# represents color channel, in this case red (R), green (G) or blue (B).
# This would make a purple version of the image:
purple_tiffany = tiffany.copy()
purple_tiffany[:, :, 1] = 0
plt.imshow(purple_tiffany)

In [ ]:
# This means that the easiest way to make a gray version of a color image
# is just to merge its channels down:
tiffany_gray = np.median(tiffany, axis=2)
plt.imshow(tiffany_gray)

In [ ]:
# We can now turn it to a black-and-white image with thresholding.
# Picking the correct threshold value is a little bit of an art and
# depends on exactly what features you want the outline to
# retain. Let's see what happens at a few different levels...
fig, axes = plt.subplots(1, 3, figsize=(10, 8))
threshold_levels = (10, 50, 160)
for i, level in enumerate(threshold_levels):
    axes[i].imshow(tiffany_gray > level)
    axes[i].set_title(level)

In [ ]:
# Let's go ahead and pick the middle value for our outline.
tiffany_outline = tiffany_gray > 50

### Morphological Operators

There are really only two morphological operators: dilation and erosion. 
Others are made up of combinations of these in different orders.

Dilation is an "or". If there is a pixel valued 1 anywhere in the dilation 
operator's footprint, it sets the center pixel to 1; otherwise, it sets it
to 0. Erosion is an "and": if _all_ pixels in the erosion operator's 
footprint are 1, it sets the center pixel to 1; otherwise, it sets the
center pixel to 0.

This means that erosion will tend to make black parts of the image heavier,
thicker, and more coherent, and dilation will do the opposite.

In [ ]:
# Just like the median filter we saw earlier, bigger footprints tend to make
# morphological operations "stronger".

# Because this particular image is basically a black-on-white outline, erosion 
# will tend to make its lines thicker and its sections more distinct:
small, big = 4, 9
fig, axes = plt.subplots(1, 2, figsize=(6, 8))
for i, size in enumerate((small, big)):
    footprint = np.ones((size, size))
    axes[i].imshow(ndi.binary_erosion(tiffany_outline, footprint))

In [ ]:
# Whereas dilation will tend to make its lines thinner and blur sections:
fig, axes = plt.subplots(1, 2, figsize=(6, 8))
for i, size in enumerate((small, big)):
    footprint = np.ones((size, size))
    axes[i].imshow(ndi.binary_dilation(tiffany_outline, footprint))

### Labeling

Being able to manipulate line like this is very useful in part because it
enables us to easily define -- and find -- contiguous regions of an image.
This process is called "labeling", and it is a powerful way to segment
images and extract regions of interest.

In [ ]:
# Let's go ahead and work with the thicker outline based on the
# size-9 footprint:
trace = ndi.binary_erosion(tiffany_outline, np.ones((9, 9)))
# The 'label' function will give every contiguous 1-valued region 
# its own unique number. Note how it cuts the sections at the 
# edges off, and can't quite distinguish some regions that might
# look contiguous to your eye because of some junky little line bits
# that connect them:
labels, n_labels = ndi.label(trace)
plt.imshow(labels, cmap='tab20', interpolation='none')

In [ ]:
# We can use this to pick out, count, and locate individual
# regions of an image. Try running this cell a few times.
# You'll note that not every label is interesting -- the algorithm
# will happily assign a unique label to even a tiny little dot.
selected = np.isin(labels, np.random.choice(np.unique(labels), 8))
plt.imshow(
    np.where(selected, labels, 0), cmap='Dark2_r', interpolation="none"
)